# AI Assistant Extensibility

This notebook presents how to extend the AI Assistant plugin in the runtime by providing a custom message handler.

In [1]:
%use intellij-platform

IntelliJ Platform integration is loaded

Load the AI Assistant bundled plugin using its `com.intellij.ml.llm` ID.

In [2]:
loadPlugins("com.intellij.ml.llm")

In [3]:
import com.intellij.ml.llm.agents.ChatAgent
import com.intellij.ml.llm.core.chat.messages.ChatMessage
import com.intellij.ml.llm.core.chat.messages.UserMessage
import com.intellij.ml.llm.core.chat.messages.impl.SimpleCompletableMessage
import com.intellij.ml.llm.core.chat.session.ChatKind
import com.intellij.ml.llm.core.chat.session.ChatMessageHandler
import com.intellij.ml.llm.core.chat.session.ChatSession
import com.intellij.ml.llm.smartChat.endpoints.SmartChatEndpoint
import com.intellij.openapi.application.ApplicationManager
import com.intellij.openapi.extensions.ExtensionPointName
import com.intellij.openapi.project.Project

class MyChatMessageHandler : ChatMessageHandler {
    override fun isApplicable(project: Project, kind: ChatKind, userMessage: UserMessage) = true

    override fun createAnswerMessage(
        project: Project,
        chat: ChatSession,
        userMessage: UserMessage,
        kind: ChatKind,
    ) = SimpleCompletableMessage(chat)

    override suspend fun serveAnswerMessage(
        project: Project,
        chat: ChatSession,
        answerMessage: ChatMessage,
        smartChatEndpoints: List<SmartChatEndpoint>,
    ) = ChatAgent.EP_NAME.extensionList.last().serveAnswerMessage(project, chat, answerMessage)
}

registerExtension(ChatMessageHandler.EP, MyChatMessageHandler())

In [4]:
import com.intellij.ml.llm.agents.ChatAgent
import com.intellij.ml.llm.core.chat.messages.ChatMessage
import com.intellij.ml.llm.core.chat.messages.impl.SimpleCompletableMessage
import com.intellij.ml.llm.core.chat.session.ChatSession
import com.intellij.ml.llm.privacy.trustedStringBuilders.privacyConst
import com.intellij.openapi.application.ApplicationManager
import com.intellij.openapi.project.Project

class MyChatAgent : ChatAgent {

    override val id = "groot"
    override val name = "I am Groot"

    override fun createAnswerMessage(
        project: Project,
        chat: ChatSession,
        userMessage: UserMessage,
        kind: ChatKind
    ): ChatMessage = SimpleCompletableMessage(chat)

    override suspend fun serveAnswerMessage(
        project: Project,
        chat: ChatSession,
        answerMessage: ChatMessage,
    ) {
        if (answerMessage !is SimpleCompletableMessage) return
        answerMessage.appendText("*I am Groot*".privacyConst)
    }
}

registerExtension(ChatAgent.EP_NAME, MyChatAgent())